In [1]:
#Install BeautifulSoup
!pip install beautifulsoup4

In [2]:
import urllib2

In [3]:
from bs4 import BeautifulSoup

In [4]:
#Open URL and read it
url = urllib2.urlopen("https://www.amazon.ca/gp/new-releases/books")
#url = urllib2.urlopen("https://www.amazon.ca/gp/bestsellers/books/ref=sv_b_3")
content = url.read()
url
#print content

<addinfourl at 152394504L whose fp = <socket._fileobject object at 0x0000000001AC1ED0>>

In [5]:
soup = BeautifulSoup(content)
# The top 100 books is divided across 5 web pages. We need to get the URL
# of those pages


links=[] #create a list of links to the 5 pages
###Moe: added the next print statement & changed the variable names to something more readable
print ("The HTML strucure for the books URLs, looks like this:\n<div id = 'zg_paginationWrapper' > \n   <a href= 'anylink' />")
for divTag in soup.find_all( "div", {"id" : "zg_paginationWrapper"}):
    for aTag in divTag.find_all('a'):
        links.append(aTag.get('href'))

links
#print links 

The HTML strucure for the books URLs, looks like this:
<div id = 'zg_paginationWrapper' > 
   <a href= 'anylink' />


C:\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


['https://www.amazon.ca/gp/new-releases/books?ie=UTF8&pg=1',
 'https://www.amazon.ca/gp/new-releases/books?ie=UTF8&pg=2',
 'https://www.amazon.ca/gp/new-releases/books?ie=UTF8&pg=3',
 'https://www.amazon.ca/gp/new-releases/books?ie=UTF8&pg=4',
 'https://www.amazon.ca/gp/new-releases/books?ie=UTF8&pg=5']

In [6]:
#Initialize dictionary to fill with data 
my_dict={}
#
for link in links:
    url = urllib2.urlopen(link)
    content = url.read()
    soup = BeautifulSoup(content)
    ###Moe: is the comment in the next line relivent? 
    #The books data is in under tag 'div' and class 'zg_itemWrapper'
    #books_data=soup.findAll("div", { "class" : "zg_itemWrapper"})
    books_data=soup.findAll("div", { "class" : "zg_itemImmersion"})
    
    #For each item tagged "zg_itemWrapper" is extracted price, author, release date and format
    for line in books_data:
        #try:
            key=line.find(class_="zg_rankNumber").get_text().encode("utf-8").rstrip('.') #the key will the rank number
            my_dict[key]={}
            my_dict[key]["Name"]=line.find(class_="zg_title").get_text().encode("utf-8")
            price=line.find(class_="price").get_text().encode("utf-8")[5:]
            my_dict[key]["Price"]=float(price)
            my_dict[key]["Author"]=line.find(class_="zg_byline").get_text().rstrip('\n')[3:].encode("utf-8")
            try:
                unformatedDate = line.find(class_="zg_releaseDate").get_text()[14:].encode("utf-8")
                #clean the returned date 
                my_dict[key]["Release Date"]=unformatedDate.replace("te:","") 
            except AttributeError:
                my_dict[key]["Release Date"]= "N/A"       
            my_dict[key]["Format"]=line.find(class_="zg_bindingPlatform").get_text().encode("utf-8")
        #except:
            #pass
            
#books_data
my_dict

{'1': {'Author': 'Connor Franta',
  'Format': 'Hardcover',
  'Name': 'Note to Self',
  'Price': 19.02,
  'Release Date': 'April 18 2017'},
 '10': {'Author': 'Gary Taubes',
  'Format': 'Hardcover',
  'Name': 'The Case Against Sugar',
  'Price': 25.66,
  'Release Date': 'Dec 27 2016'},
 '100': {'Author': 'Phil Gaimon',
  'Format': 'Hardcover',
  'Name': 'Ask a Pro: Deep Thoughts and Unreliab...',
  'Price': 23.91,
  'Release Date': ' April 2 2017'},
 '11': {'Author': 'Dr. Elizabeth Blackburn',
  'Format': 'Hardcover',
  'Name': 'The Telomere Effect: A Revolutionary...',
  'Price': 31.44,
  'Release Date': 'Jan. 3 2017'},
 '12': {'Author': 'Travis Stork',
  'Format': 'Hardcover',
  'Name': 'The Lose Your Belly Diet: Change Your...',
  'Price': 20.51,
  'Release Date': 'Dec 27 2016'},
 '13': {'Author': 'Natasha Turner',
  'Format': 'Hardcover',
  'Name': 'The Hormone Boost: How to Power Up Yo...',
  'Price': 19.2,
  'Release Date': 'Dec 27 2016'},
 '14': {'Author': 'Tim Bogenn',
  'Format'

In [7]:
import csv

In [8]:
#Write CSV file
with open("prices_2.csv", "w") as toWrite:
    writer = csv.writer(toWrite, delimiter=",")
    writer.writerow([ "Rank","Name", "Author", "Realease Date","Format", "Price"])
    rank=my_dict.keys()
    rank= map(int, rank)
    rank.sort()
    for a in rank:
        a=str(a)
        #try:
            #writer.writerow([a, my_dict[a]['Name'], my_dict[a]['Author'], my_dict[a]["Release Date"], my_dict[a]["Format"],my_dict[a]["Price"]])
        writer.writerow([a, my_dict[a]['Name'], my_dict[a]['Author'], my_dict[a]["Release Date"], my_dict[a]["Format"],my_dict[a]["Price"]])
        #except:
            #pass

In [9]:
#Import CSV file to open
import pandas
#books_prices = pandas.read_csv('/resources/prices_2.csv')

books_prices = pandas.read_csv('prices_2.csv') 
books_prices

,Rank,Name,Author,Realease Date,Format,Price
0,1,Note to Self,Connor Franta,April 18 2017,Hardcover,19.02
1,2,"Food, Health, and Happiness: 115 On-P...",Oprah Winfrey,Jan. 3 2017,Hardcover,22.83
2,3,Trumping Trudeau: How Donald Trump wi...,Ezra Levant,Jan. 17 2017,Paperback,9.50
3,4,Joyous Detox: Your Complete Plan and...,Joy McCarthy,Dec 27 2016,Paperback,20.73
4,5,Hot Detox: A 21-Day Anti-Inflammatory...,Julie Daniluk,Dec 27 2016,Paperback,18.41
5,6,Saga Volume 7,Brian K Vaughan,April 4 2017,Paperback,11.74
6,7,National Geographic Guide to the Nati...,National Geographic,Dec 27 2016,Paperback,14.82
7,8,The Legend of Zelda: Art & Artifacts,Nintendo,Feb. 21 2017,Hardcover,32.39
8,9,Dog Man Unleashed (Dog Man #2) - From...,Dav Pilkey,Dec 27 2016,Hardcover,11.69
9,10,The Case Against Sugar,Gary Taubes,Dec 27 2016,Hardcover,25.66


In [10]:
#Show Webpage
from IPython.display import HTML
HTML('https://www.amazon.ca/gp/new-releases/books')